In [ ]:
# EfficientNet-B5 문서 이미지 분류 모델
# Train 데이터: 1570장 (깨끗한 이미지)
# Test 데이터: 3140장 (회전, 플립, 노이즈가 심한 이미지)
# 클래스: 17개 (불균형 데이터)

import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ================================
# 1. Seed 고정 (재현성 확보)
# ================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [ ]:
# ================================
# 2. 설정 및 하이퍼파라미터
# ================================
class Config:
    # 경로
    train_csv = '/home1/dg5407/cv_project/data/train.csv'
    meta_csv = '/home1/dg5407/cv_project/data/meta.csv'
    train_img_dir = '/home1/dg5407/cv_project/data/train'
    test_img_dir = '/home1/dg5407/cv_project/data/test'
    submission_csv = '/home1/dg5407/cv_project/data/sample_submission.csv'
    
    # 모델 설정
    model_name = 'tf_efficientnet_b5'  # EfficientNet-B5
    img_size = 456  # EfficientNet-B5의 기본 입력 크기
    num_classes = 17
    
    # 학습 설정
    n_folds = 5
    epochs = 50
    batch_size = 16  # B5는 메모리를 많이 사용하므로 조정 필요
    num_workers = 48
    
    # Learning Rate 설정 (warmup + cosine annealing)
    max_lr = 1e-3  # 초기 높은 learning rate
    min_lr = 1e-6  # 최종 낮은 learning rate
    warmup_epochs = 5  # warmup 기간
    
    # 기타
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    use_wandb = True
    project_name = 'document-classification'
    
config = Config()

In [ ]:
# ================================
# 3. WandB 초기화
# ================================
if config.use_wandb:
    wandb.login(key='API_KEY')
    wandb.init(
        project=config.project_name,
        name=f'efficientnet_b5_fold_all',
        config={
            'model': config.model_name,
            'img_size': config.img_size,
            'epochs': config.epochs,
            'batch_size': config.batch_size,
            'max_lr': config.max_lr,
            'min_lr': config.min_lr,
            'warmup_epochs': config.warmup_epochs,
            'n_folds': config.n_folds,
        }
    )

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home1/dg5407/.netrc
